In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd

import efd_utils

In [ ]:
location = "summit"

In [ ]:
client = efd_utils.get_client(location)

In [ ]:
csc = efd_utils.CSC("Environment", 1)
d1 = datetime(2019, 9, 16)
d2 = datetime(2019, 9, 23)
summary_state = 3  # FAULT
time_window = 1.0  # seconds

In [ ]:
query = efd_utils.get_base_query(columns=["private_sndStamp",
                                          "summaryState"],
                                 csc_name=csc.name,
                                 csc_index=csc.index,
                                 topic_name="logevent_summaryState")
# Handle indexed components from base query
if "WHERE" not in query:
    query += " WHERE"
query += f" AND summaryState={summary_state}"
query += " AND " + efd_utils.get_time_clause(date_range=(d1, d2))
ss_df = await client.query(query)

In [ ]:
query = efd_utils.get_base_query(columns=["private_sndStamp",
                                          "errorCode",
                                          "errorReport",
                                          "traceback"],
                                 csc_name=csc.name,
                                 csc_index=csc.index,
                                 topic_name="logevent_errorCode")
query += " AND " + efd_utils.get_time_clause(date_range=(d1, d2))
ec_df = await client.query(query)

In [ ]:
timedelta = pd.Timedelta(time_window, unit='sec')

In [ ]:
if ss_df.index.size:
    for timestamp in ss_df.index:
        t1 = timestamp - timedelta
        t2 = timestamp + timedelta
        error_indexes = ec_df.index.to_numpy()
        index = np.logical_and(error_indexes > t1, error_indexes < t2)
        error = ec_df[index]
        print("######################################")
        print(f"Fault State: {error.errorCode.index.strftime('%Y-%m-%dT%H:%M:%S.%f')[0]}")
        print(f"Error Code: {error.errorCode.values[0]}")
        print(f"Error Report: {error.errorReport.values[0]}")
        print(f"Traceback: {error.traceback.values[0]}")
else:
    print(f"No fault states found for {csc}")